In [1]:
import duckdb
from os import remove

SF = 1

duckdb.__version__

'v0.7.1-dev648'

In [4]:
database_file = "snb.duckdb"

con = duckdb.connect(database_file)
schema = open(f"SNB-SF{SF}-dataset/minimal_schema.sql").read().split(";")
for query in schema:
    con.execute(query)
load = open(f"SNB-SF{SF}-dataset/minimal_load.sql").read().split(";")
for query in load:
    con.execute(query)
print(f"Loaded dataset of scale factor {SF}")

con.close()

Loaded dataset of scale factor 1


In [5]:
%load_ext sql


There's a new jupysql version available (0.6.6), you're running 0.7.0.dev0. To upgrade: pip install jupysql --upgrade


In [7]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///snb.duckdb

%sql load 'sqlpgq'

""


In [8]:
%%sql
    CREATE PROPERTY GRAPH social_network
    VERTEX TABLES (
        Person LABEL Person
    )
    EDGE TABLES (
        Person_knows_Person 
            SOURCE KEY (Person1id) REFERENCES Person (id)
            DESTINATION KEY (Person2id) REFERENCES Person (id)
            LABEL Person_knows_Person
    )

""


In [ ]:
%%sql
    SELECT pg.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->{1,4}(p2:person)
    WHERE p.firstname = 'Bob'
    COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
    ) pg
    GROUP BY ALL

In [ ]:
%%sql
            SELECT pg.firstname, pg.id, count(pg.reachable_people)
            FROM GRAPH_TABLE (social_network,
            MATCH (p:Person)-[k:Person_knows_Person]-> *(p2:person)
            WHERE p.firstname = 'Bob'
            COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
            ) pg
            GROUP BY ALL